In [36]:
# Dependencies and Setup
import pandas as pd
import numpy as np


# Raw data file
file_to_load = "Resources/nfl_data.csv"

# Read purchasing file and store into pandas data frame
nfldata_raw = pd.read_csv(file_to_load)

In [47]:
#Grabbing Columns relavent to our research
nfl = nfldata_raw[["Date", "GameID", "down", "Yards.Gained", "FirstDown", "Touchdown", "PlayType", \
                  "PassOutcome", "PassLength", "AirYards", "PassLocation", "RushAttempt", \
                  "RunLocation", "Reception", "Season"]]

#creating a column to count play for future sumations
nfl["PlayCount"] = 1

#dropping rows where [down]=Nan; these represent special teams plays and extra points
nfl.dropna(subset=["down"], inplace=True)

#filtering for only playtypes = Pass or Run
nfl = nfl[(nfl["PlayType"] == "Pass") | (nfl["PlayType"] =="Run")]
nfl.count()
#Dropping Where Play type = Run AND Run Location is NA
#Dropping Where Play type = Pass AND Pass Location is NA

C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Date            279552
GameID          279552
down            279552
Yards.Gained    279552
FirstDown       279552
Touchdown       279552
PlayType        279552
PassOutcome     158929
PassLength      158435
AirYards        279552
PassLocation    158435
RushAttempt     279552
RunLocation     119303
Reception       279552
Season          279552
PlayCount       279552
dtype: int64

In [38]:
#Blank Lists used to append
ptotals = []
p = []
rtotals = []
r = []

In [40]:
#NFL Passing Data
#Grabbing only the records where the PlayType = Pass from our NFL Data
nfl_pass = nfl.loc[nfl['PlayType'] == "Pass"]

#dropping where passlocations = NA
nfl_pass.dropna(subset=["PassLocation"], inplace=True)

#Create DataFrame for only Passing columns
npass = nfl_pass[["down", "Yards.Gained", "AirYards", "FirstDown", "Touchdown", "PlayCount"]]

#Group by Down
p = npass.groupby(["down"]).sum()

#Create DataFrame of Totals for all downs
ptotals = pd.DataFrame({"down": ["Totals"], 
                       "Yards.Gained": p["Yards.Gained"].sum(),
                       "AirYards": p["AirYards"].sum(),
                       "FirstDown": p["FirstDown"].sum(),
                       "Touchdown": p["Touchdown"].sum(),
                       "PlayCount": p["PlayCount"].sum()})
ptotals = ptotals.set_index("down")
#Append Totals to Pass table
p = p.append(ptotals)

#Average/Rates
p['AvgYardsGained_PerPlay'] = round(p["Yards.Gained"] / p["PlayCount"], 3)
p['RateofFirstDown_PerPlay'] = round(p["FirstDown"] / p["PlayCount"], 3)
p['RateofTouchdown_PerPlay'] = round(p["Touchdown"] / p["PlayCount"], 3)

p

C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Yards.Gained,AirYards,FirstDown,Touchdown,PlayCount,AvgYardsGained_PerPlay,RateofFirstDown_PerPlay,RateofTouchdown_PerPlay
down,,,,,,,,
1.0,446962,512009,15982.0,2604,59123,7.560,0.270,0.044
2.0,366677,401732,16249.0,2410,52671,6.962,0.308,0.046
3.0,311825,387300,15534.0,2373,44119,7.068,0.352,0.054
4.0,15695,26543,2091.0,238,2521,6.226,0.829,0.094
Totals,1141159,1327584,49856.0,7625,158434,7.203,0.315,0.048


In [45]:
#Pass Breakdowns

#Create DataFrame for only Passing columns
npass = nfl_pass[["down", "Yards.Gained", "AirYards", "FirstDown", "Touchdown", "PlayCount", "PassLocation", "PassOutcome"]]
npass["CompletedPassCount"] = np.where(npass['PassOutcome']=='Complete', 1, 0)

#Group by Down
pb = npass.groupby(["down", "PassLocation"]).sum()

#Completion Rates
pb["PassCompletionRate"] = round(pb["CompletedPassCount"] / pb["PlayCount"], 3)
pb

C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Yards.Gained  AirYards  FirstDown  Touchdown  PlayCount  \
down PassLocation                                                            
1.0  left                158420    180009     5624.0        921      21441   
     middle              119690    133818     4402.0        676      13188   
     right               168852    198182     5956.0       1007      24494   
2.0  left                131116    142118     5917.0        819      19337   
     middle               96505    106269     4159.0        646      11892   
     right               139056    153345     6173.0        945      21442   
3.0  left                104568    137009     5294.0        792      15654   
     middle               94153    103489     4476.0        705      11414   
     right               113104    146802     5764.0        876      17051   
4.0  left                  5042      8790      699.0         81        828   
     middle                5216      7006      556.0         63        677   
     right                 5437     10747      836.0         94       1016   

                   CompletedPassCount  PassCompletionRate  
down PassLocation                                          
1.0  left                       13633               0.636  
     middle                      8734               0.662  
     right                      15257               0.623  
2.0  left                       12404               0.641  
     middle                      7723               0.649  
     right                      13467               0.628  
3.0  left                        8888               0.568  
     middle                      7093               0.621  
     right                       9579               0.562  
4.0  left                         415               0.501  
     middle                       377               0.557  
     right                        496               0.488

In [41]:
#NFL Rushing Data
#Grabbing only the records where the PlayType = Pass from our NFL Data
nfl_run = nfl.loc[nfl['PlayType'] == "Run"]

#dropping where runlocations = NA
nfl_run.dropna(subset=["RunLocation"], inplace=True)

#Create DataFrame for only Passing columns
nrun = nfl_run[["down", "Yards.Gained", "RushAttempt", "FirstDown", "Touchdown", "PlayCount"]]

#Group by Down
r = nrun.groupby(["down"]).sum()

#Create DataFrame of Totals for all downs
rtotals = pd.DataFrame({"down": ["Totals"], 
                       "Yards.Gained": r["Yards.Gained"].sum(),
                       "RushAttempt": r["RushAttempt"].sum(),
                       "FirstDown": r["FirstDown"].sum(),
                       "Touchdown": r["Touchdown"].sum(),
                       "PlayCount": r["PlayCount"].sum()})
rtotals = rtotals.set_index("down")
#Append Totals to Pass table
r = r.append(rtotals)

#Average/Rates
r['AvgYardsGained_PerPlay'] = round(r["Yards.Gained"] / r["PlayCount"], 3)
r['RateofFirstDown_PerPlay'] = round(r["FirstDown"] / r["PlayCount"], 3)
r['RateofTouchdown_PerPlay'] = round(r["Touchdown"] / r["PlayCount"], 3)

r

C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Yards.Gained,RushAttempt,FirstDown,Touchdown,PlayCount,AvgYardsGained_PerPlay,RateofFirstDown_PerPlay,RateofTouchdown_PerPlay
down,,,,,,,,
1.0,285121,64984,7098.0,1724,64984,4.388,0.109,0.027
2.0,177259,40467,10285.0,1251,40467,4.380,0.254,0.031
3.0,58182,12368,5313.0,611,12368,4.704,0.430,0.049
4.0,4889,1427,1205.0,135,1427,3.426,0.844,0.095
Totals,525451,119246,23901.0,3721,119246,4.406,0.200,0.031


In [49]:
#Run Breakdowns

#Create DataFrame for only Rushing columns
nrun = nfl_run[["down", "Yards.Gained", "FirstDown", "Touchdown", "PlayCount", "RunLocation"]
#nrun["CompletedPassCount"] = np.where(npass['PassOutcome']=='Complete', 1, 0)

#Group by Down
rb = nrun.groupby(["down", "RunLocation"]).sum()

#Completion Rates
#pb["PassCompletionRate"] = round(pb["CompletedPassCount"] / pb["PlayCount"], 3)
rb

SyntaxError: invalid syntax (<ipython-input-49-f8416a3e3952>, line 8)